In [370]:
# 환경셋팅
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV, Ridge, Lasso

In [372]:
# pip install threadpoolctl==3.1.0

In [374]:
# 행, 열 10개로 제한해서 보기
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)

In [376]:
# 데이터를 불러와서 data 변수에 담기
# data = pd.read_csv('./data/games_details.csv')
data = pd.read_csv('./archive/games_details.csv')#내 컴 경로

C:\Users\USER\AppData\Local\Temp\ipykernel_4024\1443809302.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./archive/games_details.csv')#내 컴 경로


In [377]:
# 필요없는 컬럼 삭제
data.drop(['NICKNAME','START_POSITION','COMMENT'], axis = 1, inplace=True)
data.drop('PLUS_MINUS', axis = 1, inplace=True)
data.drop(['PLAYER_ID','PLAYER_NAME','MIN','TEAM_ABBREVIATION'], axis = 1, inplace=True)

In [378]:
# 행 결측치 삭제
data.dropna(axis=0, inplace=True)

In [379]:
# 데이터 그룹화 및 컬럼값 재정의
data1 = data.groupby(['GAME_ID','TEAM_ID','TEAM_CITY']).sum()
data1['FG_PCT'] = round(data1['FGM'] / data1['FGA'],3)
data1['FG3_PCT'] = round(data1['FG3M'] / data1['FG3A'],3)
data1['FT_PCT'] = round(data1['FTM'] / data1['FTA'],3)

In [380]:
# data1['perc_2'] = round(2*data1['FGA'] / (2*data1['FGA'] + 3*data1['FG3A'] + data1['FTA']),3)

In [386]:
# data1['perc_3'] = 3*data1['FG3A'] / (2*data1['FGA'] + 3*data1['FG3A'] + data1['FTA'])

In [388]:
# data1['perc_1'] = data1['FTA'] / (2*data1['FGA'] + 3*data1['FG3A'] + data1['FTA'])

In [390]:
# sorted(data['GAME_ID'].unique())

In [392]:
# FGM, FGA 컬럼을 FG2M, FG2A 컬럼으로 세분화시키기
data1['FG2M'] = data1['FGM'] - data1['FG3M']
data1['FG2A'] = data1['FGA'] - data1['FG3A']
data1['FG2_PCT'] = round(data1['FG2M'] / data1['FG2A'],3)
data2 = data1.T

In [394]:
# 경기 ID를 기준으로 데이터 재정의
data4 = {}
for i,j in enumerate(sorted(data['GAME_ID'].unique())):
    data3 = data2.iloc[:,2*i] - data2.iloc[:,2*i+1]
    data4[j] = data3

In [395]:
# 데이터를 데이터프레임화시키고 결측치 제거 (데이터:시리즈들이 담겨진 딕셔너리)
data5 = pd.DataFrame(data4).T
data5.dropna(axis=0, inplace= True)

#### 컬럼들 라벨링화 하기
#### ※ 데이터간 중요 속성 찾아보기를 참고

In [397]:
data5['y0'] = pd.cut(data5['PTS'], bins=[-100,0,100], labels=['L','W'])

In [398]:
data5['y_xgb'] = pd.cut(data5['PTS'], bins=[-100,0,100], labels=[0,1])

In [399]:
# data5['y1'] = pd.cut(data5['PTS'], bins=[-100,-5,0,5,100], labels=['L','CL','CW','W'])

In [400]:
# data5['y2'] = pd.cut(data5['PTS'], bins=[-100,-5,5,100], labels=['L','C','W'])

In [401]:
# data5['y2'] = pd.cut(data5['PTS'], bins=[-100,-5,5,100], labels=[0,1,2])

In [409]:
data5.columns

Index(['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'FG2M', 'FG2A', 'FG2_PCT', 'y0', 'y_xgb'],
      dtype='object')

In [411]:
data5['fg_pct_y0'] = pd.cut(data5['FG_PCT'], bins=[-0.5,-0.1, -0.07, 0.07,0.1, 0.5], labels=['big_minus','minus','little','plus','big_plus']) #labels=[-2,-1,0,1,2]) 

In [413]:
data5['fgm_y0'] = pd.cut(data5['FGM'], bins=[-50,-10, -6, 6,10, 50], labels=['big_minus','minus','little','plus','big_plus']) #labels=[-2,-1,0,1,2])

In [415]:
# data5['fga_y0'] = pd.cut(data5['FGA'], bins=[-100,-30, -15, 15,30, 100], labels=['big_minus','minus','little','plus','big_plus'])

In [417]:
data5['fg3m_y0'] = pd.cut(data5['FG3M'], bins=[-30,-10, -3, 3,10, 30], labels=['big_minus','minus','little','plus','big_plus']) #labels=[-2,-1,0,1,2])

In [419]:
# data5['fg3a_y0'] = pd.cut(data5['FG3A'], bins=[-50,-20, -5,5,20, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [421]:
data5['fg3_pct_y0'] = pd.cut(data5['FG3_PCT'], bins=[-1,-0.3, -0.12, 0.12,0.3, 1], labels=['big_minus','minus','little','plus','big_plus']) #labels=[-2,-1,0,1,2])

In [423]:
# data5['ftm_y0'] = pd.cut(data5['FTM'], bins=[-50,-20, -6, 6,20, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [425]:
data5['ast_y0'] = pd.cut(data5['AST'], bins=[-50,-20, -5, 5,20, 50], labels=['big_minus','minus','little','plus','big_plus']) #labels=[-2,-1,0,1,2])

In [427]:
# data5['reb_y0'] = pd.cut(data5['REB'], bins=[-50,-25, -3, 3,25, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [429]:
# data5['dreb_y0'] = pd.cut(data5['DREB'], bins=[-50,-15, -7, 7,15, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [431]:
# data5['oreb_y0'] = pd.cut(data5['OREB'], bins=[-50,-15, -3, 3,15, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [433]:
# data5['to_y0'] = pd.cut(data5['TO'], bins=[-50,-15, -4, 4,15, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [435]:
# data5['to_y0'] = pd.cut(data5['TO'], bins=[-50,-15, -4, 4,15, 50], labels=['big_minus','minus','little','plus','big_plus'])

In [437]:
# data5['TO'].max()

#### 데이터간 중요 속성 찾아보기(시각화)
- 승패를 구별해줄 수 있는 특정 컬럼들 찾아보기

In [440]:
# sns.countplot(data5,hue='y0', x='oreb_y0')
# plt.xticks()
# plt.show()

In [442]:
# data5['y'].unique()

In [444]:
# data5['y2'].unique()

In [446]:
# data5[['PTS','y','y2']]

In [448]:
# X = data5.drop(['PTS','y','y2'], axis = 1)
# y = data5['y2']

In [450]:
lst = {}
lst.append(3)

AttributeError: 'dict' object has no attribute 'append'

#### 모델 학습 시작

In [453]:
X = data5[['fg_pct_y0','fgm_y0','fg3m_y0','fg3m_y0','ast_y0']]
y = data5['y0']

In [455]:
# y_xgb = data5['y_xgb']

In [457]:
# data_test[data_test > 0] = 1
# data_test[data_test < 0] = 0

In [459]:
# data5['PTS'].unique()

In [461]:
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state = 5)

In [463]:
# X_train, X_test, y_train_xgb, y_test_xgb = train_test_split(X, y_xgb , random_state = 5)

In [465]:
# One-Hot Encoding 적용-get_dummies(): 숫자형 값 변환 없이 바로 변환 가능
X_train = pd.get_dummies(X_train)
X_train
X_test = pd.get_dummies(X_test)
X_test

,fg_pct_y0_big_minus,fg_pct_y0_minus,fg_pct_y0_little,fg_pct_y0_plus,fg_pct_y0_big_plus,...,ast_y0_big_minus,ast_y0_minus,ast_y0_little,ast_y0_plus,ast_y0_big_plus
20900086,False,False,True,False,False,...,False,False,True,False,False
21600246,False,False,True,False,False,...,False,False,False,True,False
20601066,False,False,False,True,False,...,False,False,True,False,False
11000085,False,False,False,False,True,...,False,False,True,False,False
20300276,False,False,True,False,False,...,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
42000224,False,False,True,False,False,...,False,False,True,False,False
11300053,False,False,True,False,False,...,False,True,False,False,False
20600265,False,False,True,False,False,...,False,False,True,False,False
21000518,False,False,True,False,False,...,False,False,True,False,False


In [467]:
# sns.countplot(data=data_train, x='FGM', hue='PTS')

In [469]:
# X_train.drop(['FGM', 'FGA','FG3M', 'FG3A','FTM', 'FTA','FG2M','FG2A'], axis = 1, inplace=True)
# X_test.drop(['FGM', 'FGA','FG3M', 'FG3A','FTM', 'FTA','FG2M','FG2A'], axis = 1, inplace=True)

In [471]:
# X_train1 = X_train.drop(['FGM','FG3M','FTM','FG2M','FGA','FG3A','FG2A','FTA'], axis = 1)
# X_test1 = X_test.drop(['FGM','FG3M','FTM', 'FG2M','FGA','FG3A','FG2A','FTA'], axis = 1)

#### knn_model 학습

In [474]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [476]:
# cross_val_score(knn_model,X_train1,y_train, cv=5)
cross_val_score(knn_model,X_train,y_train, cv=5)

array([0.75270168, 0.75992961, 0.72750126, 0.77325289, 0.72121669])

In [477]:
pre = knn_model.predict(X_test)

In [478]:
accuracy_score(y_test, pre)

0.7665510481073744

In [479]:
# 결과 출력
print(f"교차 검증 점수: {cross_val_score(knn_model,X_train,y_train, cv=5)}")
print(f"평균 교차 검증 점수: {cross_val_score(knn_model,X_train,y_train, cv=5).mean()}")
print(f"테스트 세트 정확도: {accuracy_score(y_test, pre)}")


교차 검증 점수: [0.75270168 0.75992961 0.72750126 0.77325289 0.72121669]
평균 교차 검증 점수: 0.7469204272657698
테스트 세트 정확도: 0.7665510481073744


#### tree_model 학습

In [486]:
# tree_model = DecisionTreeClassifier(max_depth=11, min_samples_leaf=40, min_samples_split=100)

In [488]:
tree_model = DecisionTreeClassifier()

In [490]:
tree_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [492]:
cross_val_score(tree_model, X_train, y_train, cv=5)

array([0.77632571, 0.79110106, 0.77828054, 0.78154852, 0.79587733])

In [494]:
pre1 = tree_model.predict(X_test)

In [496]:
accuracy_score(y_test, pre1)

0.7885688433117177

In [498]:
# 결과 출력
print(f"교차 검증 점수: {cross_val_score(tree_model,X_train,y_train, cv=5)}")
print(f"평균 교차 검증 점수: {cross_val_score(tree_model,X_train,y_train, cv=5).mean()}")
print(f"테스트 세트 정확도: {accuracy_score(y_test, pre1)}")


교차 검증 점수: [0.77632571 0.79135244 0.77828054 0.78154852 0.79587733]
평균 교차 검증 점수: 0.784576353659629
테스트 세트 정확도: 0.7885688433117177


#### rf_model 학습

In [501]:
rf_model = RandomForestClassifier()

In [503]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [505]:
cross_val_score(rf_model, X_train, y_train, cv=5)

array([0.77607439, 0.79034691, 0.77727501, 0.7817999 , 0.79587733])

In [507]:
pre2 = rf_model.predict(X_test)

In [509]:
accuracy_score(y_test, pre2)

0.7888704569446539

In [511]:
# 결과 출력
print(f"교차 검증 점수: {cross_val_score(rf_model,X_train,y_train, cv=5)}")
print(f"평균 교차 검증 점수: {cross_val_score(rf_model,X_train,y_train, cv=5).mean()}")
print(f"테스트 세트 정확도: {accuracy_score(y_test, pre2)}")


교차 검증 점수: [0.77632571 0.79034691 0.77727501 0.7817999  0.79587733]
평균 교차 검증 점수: 0.7846769067013585
테스트 세트 정확도: 0.7888704569446539


In [352]:
# plt.figure(figsize=(40,15))
# sns.countplot(data=data5, x="FGM", hue='PTS')

In [354]:
# data5

In [356]:
# from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

#### 최적의 tree_model의 하이퍼 파라미터 찾기

In [359]:
# # 1. 튜닝할 매개변수 정의
# # max_depth
# # min_samples_split
# # min_samples_leaf
# # max_leaf_nodes
# params = { #'max_depth':range(3,10)
#         'max_depth':[9,11,13,15,17,19],
#          'min_samples_split':[30,40,50,60,70,80,90,100],
#           'min_samples_leaf':[2,3,5,10,20,30,40,50,60],
#           # 'max_leaf_nodes':[7,9,11,13,15,17,19,21]
#          }

# tree = DecisionTreeClassifier()

# grid = GridSearchCV(tree,params,cv=5, scoring = 'accuracy', n_jobs=-1,verbose = 1)

# # 여러 조합을 모델에 연결하며 훈련을 진행 -> 최적의 파라미터 조합 알려줌
# grid.fit(X_train1,y_train)

In [361]:
# print(grid.best_params_)
# print(grid.best_score_) # 0.34 - 35% 성능

#### 최적의 tree_model에 따른 시각화

In [364]:
# from sklearn.tree import export_graphviz
# export_graphviz(tree_model, out_file='tree.dot',
#                class_names=['0','1'],
#                feature_names=X_train1.columns,
#                impurity=False,
#                filled=True)

# import graphviz

# with open('tree.dot', encoding='UTF8') as f:
#     dot_graph = f.read()

# display(graphviz.Source(dot_graph))

In [366]:
# from sklearn.tree import export_graphviz
# import graphviz

# # 결정 트리 모델을 .dot 파일로 내보내기
# export_graphviz(tree_model, out_file='tree.dot',
#                class_names=['lose', 'win'],
#                feature_names=X_train1.columns,
#                impurity=False,
#                filled=True)

# # .dot 파일 읽기
# with open('tree.dot', encoding='UTF8') as f:
#     dot_graph = f.read()

# # 그래프 크기 조정
# graph = graphviz.Source(dot_graph, format='png')
# graph.render('tree')  # 이미지를 파일로 저장

# # Jupyter Notebook에서 그래프 크기 조정
# graph.view()
